**Section 1: Install required libraries**

In [1]:
! pip install requests
! pip install ollama
! pip install duckdb
! pip install chromadb
! pip install langchain
! pip install langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 4.8 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable


**Section 2: Load Data**

For this demo we will be using the discharge summaries for the new born kids diagnosed with the Jaundice disease

The data required for our processing is extracted from the MIMIC-III database using the below query and then saved as a CSV file named NOTEEVENTS_EXTRACT.csv which we will be using for processing 

CREATE TEMPORARY TABLE NOTEEVENTS_EXTRACT AS
SELECT P.SUBJECT_ID, ADMISSION_TYPE, CATEGORY, ICD9_CODE, TEXT, LEN(TEXT) as CNT
FROM ADMISSIONS A
INNER JOIN PATIENTS P ON A.SUBJECT_ID = P.SUBJECT_ID
INNER JOIN NOTEEVENTS N ON P.SUBJECT_ID = N.SUBJECT_ID
INNER JOIN DIAGNOSES_ICD D ON D.SUBJECT_ID = N.SUBJECT_ID
WHERE DIAGNOSIS = 'NEWBORN' AND ADMISSION_TYPE = 'NEWBORN' AND CATEGORY = 'Discharge summary'
AND ICD9_CODE IN ['7740', '7741', '7742', '77430', '77431', '77439', '7744', '7745', '7746']
ORDER BY CNT DESC;

In [12]:
import duckdb

con = duckdb.connect()

# Create a temporary view of the csv file that we have extracted using the steps explained previously.
con.execute("""
    CREATE TEMPORARY VIEW NOTEEVENTS_EXTRACT AS
    SELECT * FROM
    read_csv_auto('NOTEEVENTS_EXTRACT.csv')
""")

# Read from the CSV and save the data in a parquet file for further processing.
new_table = 'NOTEEVENTS_EXTRACT.parquet'
con.execute(f"""
    COPY (SELECT * FROM NOTEEVENTS_EXTRACT)
    TO '{new_table}' (FORMAT PARQUET)
""")

# store the data in a dataframe
df = con.execute(f"SELECT * FROM '{new_table}'").df()

discharge_summary = df['TEXT'][2927]

In [13]:
discharge_summary = discharge_summary.replace('\n', ' ').replace('\r', ' ').replace('  ', ' ')
discharge_summary

"Admission Date: [**2180-10-25**]    Discharge Date: [**2180-10-28**] Date of Birth:  [**2180-8-2**]    Sex: M Service: NEONATOLOGY  HISTORY: Please see previous comprehensive summary dictated [**2180-10-25**], for history. This summary covers the period from [**2180-10-25**], through [**2180-10-28**]. Briefly, [**Known lastname **] [**Known lastname 43835**] is a former 27 and [**1-10**] week gestation infant, now day of life 88, who returned from [**Hospital3 1810**] status post herniorrhaphy. His postoperative course has been uneventful. He received Tylenol for pain control. He continues to breast feed or ad lib p.o. feeds. He is being discharged home on nasal cannula oxygen 50 cc/minute. Discharge weight is 3.345 kilograms. centimeters. Please see previous summary for list of medications and follow-up. DISCHARGE DISPOSITION: Home with parents. PRIMARY PEDIATRICIAN: Dr. [**First Name8 (NamePattern2) **] [**Last Name (NamePattern1) 35852**], [**Location (un) 43837**], [**Location (un

In [5]:
import ollama
import chromadb

documents = df['TEXT'].tolist()

client = chromadb.Client()
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(documents):
  if i >= 3:
    break
  # Use the Ollama API to get the embeddings for the document
  response = ollama.embed(model="all-minilm:latest", input=d)
  embeddings = response["embeddings"]
  collection.add(
    ids=[str(i)],
    embeddings=embeddings,
    documents=[d]
  )

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate


prompt_template = [
("system", """You are a seasoned professor in medicine. Your job is to evaluation your junior doctors based on the discharge summary 
of the patients to the patients. You have access to the following discharge summary. 
Use them to ask 10 questions to evaluate if the junior doctor has the proper understanding of the patient you both worked on.
Tell the junior doctor to answer the questions one by one by simple words.
The junior doctor will answer the questions one by one.
At the end of the conversation, you will give a score to the junior doctor in the scale of 1 - 10 based on the answers he provided.
Providing a score of 1 means the junior doctor has no understanding of the patient and providing a score of 10 means the junior doctor has a perfect understanding of the patient.
The junior doctor will be evaluated based on the answers he provided to the questions you asked him or her.
Provide the junior doctor with your reasoning behind the score you provided him or her.
You are not allowed to ask any questions to the junior doctor that are not related to the discharge summary of the patient.
Discharge Summary: {discharge_summary}
Here is the convestation history: {context}"""),
("human", "Let's start the evaluation."),
("ai", "Ok. Let's start the evaluation."),
("ai", "Please answer the questions one by one."),
("human", "{user_input}"),
]

model = OllamaLLM(model="llama3:8b")#, temperature=0.1, max_tokens=512)
prompt = ChatPromptTemplate.from_messages(prompt_template)
chain = prompt | model

def hadle_conversation():
    context = ""
    # Initialize the conversation with a greeting
    print("Welcome to the evaluation. Are you ready? Please type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        result = chain.invoke({"context": context, "discharge_summary": discharge_summary, "user_input": user_input})
        print("Professor:", result)
        context += f"\nUser: {user_input}\nProfessor: {result}"
        # Store the context in the database for future reference
        # Note: In a real-world application, you would want to store the context in a database or file for future reference.

In [15]:
hadle_conversation()

Welcome to the evaluation. Are you ready? Please type 'exit' to end the conversation.
Professor: I'll ask the first question, and you can respond with a simple "yes" or "no".

Here's the first question:

1. Is the patient a male?

Please respond with a simple "yes" or "no".
Professor: Good, good! So you answered "yes", indicating that the patient is indeed a male.

Here's the next question:

2. Is the patient's primary diagnosis related to prematurity?

Please respond with a simple "yes" or "no".
Professor: Let's continue the evaluation.

Since you answered "yes", I'll proceed to the next question.

Here it is:

3. Did the patient undergo herniorrhaphy during their admission?

Please respond with a simple "yes" or "no".
Professor: I'll continue the conversation.

Here's the next question:

4. Is the patient being discharged on nasal cannula oxygen?

Please respond with a simple "yes" or "no".
Professor: I'll continue the conversation.

Since you answered "no", I'll proceed to the next 